In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive/')
%cd 'drive/Shareddrives/CSCI 461'
%ls processed/*

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import os
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
[Errno 2] No such file or directory: 'drive/Shareddrives/CSCI 461'
/content/drive/Shareddrives/CSCI 461
 processed/cohort_miss.csv
 processed/combined_ohe_17.csv
 processed/combined_ohe.csv
 processed/condacolab_install.log
 processed/data_with_dummies_zoe.csv
 processed/data.zip
'processed/Feature Selection + Exploration (Services)'
 processed/gz_2010_us_040_00_5m.json
 processed/processed_data_zoe.csv
 processed/recnumbr_services_2011.csv
 processed/recnumbr_services_2014.csv
 processed/recnumbr_services_2017.csv
 processed/recnumbr_services_2020.csv
'processed/Services Use Viz (Attempt)'
 processed/stormhistory.csv
 processed/us_states.shp
 processed/us_states.shx

processed/data:
ne_110m_admin_0_countries.cpg          ne_110m_admin_0_countries.shp
ne_110m_admin_0_countries.dbf          ne_110m_admin_0_countries.shx
ne_110m_admin_0_countries.prj         

In [ ]:
COLOR_BLUEISH = '#5A9'
cmap_blueish=sns.light_palette(COLOR_BLUEISH, as_cmap=True)

In [ ]:
# services = pd.read_csv('processed/services.csv')
fostercare = pd.read_csv('processed/fostercare.csv')
merged = pd.read_csv('processed/fostercare.services.csv')
merged.columns

FileNotFoundError: ignored

In [ ]:
# services.head(1)

In [ ]:
merged['num_surveyed'].median()

In [ ]:
fig = plt.figure(figsize =(7, 2))
ax=sns.boxplot(data=merged, x='num_surveyed', color=COLOR_BLUEISH)
# ax.set(xlabel='Number of Surveys per Participant')
ax.set_title('Number of Surveys per Participant')
plt.savefig('img/02_num_surveys.png', bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize =(7, 10))

grouped = merged.loc[:,['st', 'num_surveyed']] \
    .groupby(['st']) \
    .median() \
    .sort_values(by='num_surveyed')
    
ax=sns.boxplot(data=merged, x='num_surveyed', y='st', order=grouped.index[-20:], palette="light:"+COLOR_BLUEISH)
ax.set(xlabel='Number of Surveys per Participant', ylabel='State')
ax.set_title('Top 20 States by Median Number of Surveys per Participant')
plt.savefig('img/03_states_by_num_surveys.png', bbox_inches='tight')


In [ ]:
print(set(merged['edlevlsv']))
# print(education)
educationNumerical = \
{   'Less than 6th grade' : 0,
    '6th grade' : 6,
    '7th grade' : 7,
    '8th grade' : 8,
    '9th grade' : 9,
    '10th grade' : 10,
    '11th grade' : 11,
    '12th grade' : 12,
    'Post secondary' : 13,
    'College' : 14,
    '' : 77
}
educationNumerical = \
{   'Less than 6th grade' : 0,
    '6th grade' : 1,
    '7th grade' : 1,
    '8th grade' : 1,
    '9th grade' : 2,
    '10th grade' : 2,
    '11th grade' : 2,
    '12th grade' : 2,
    'Post secondary' : 3,
    'College' : 4,
    '' : -1
}
merged['edlevlsv_numerical'] = merged['edlevlsv'].map(educationNumerical)
merged['edlevlsv_numerical']

In [ ]:
len(merged.index)

In [ ]:
ax = sns.countplot(merged['edlevlsv'], order=educationNumerical.keys(), color=COLOR_BLUEISH)
ax.set(xlabel='Education Level', ylabel='Count', title='Count of Education Level')
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.savefig('img/04_education_level_counts.png', bbox_inches='tight')

In [ ]:
# print(set(fostercarestatus))
# print(fostercarestatus)
#need to make binary
print("OneHotEncoder of fostercarestatus:")
enc = OneHotEncoder(categories='auto', handle_unknown='ignore')
transformed = enc.fit_transform(merged['fcstatsv'].to_numpy().reshape(-1,1))
enc.categories_
merged[enc.categories_[0]] = transformed.toarray()
merged.head(1)

In [ ]:
print(set(merged['outcmrpt']))
#need to make numerical like in codebook
enc2 = OneHotEncoder(categories='auto', handle_unknown='ignore')
transformed2 = enc2.fit_transform(merged['outcmrpt'].to_numpy().reshape(-1,1))
enc2.categories_
pd.set_option('display.max_columns', None)
merged[enc2.categories_[0]] = transformed2.toarray()
merged['dob.1_dt'] = pd.to_datetime(merged['dob.1'], format="%Y-%m-%d")
merged.head()


In [ ]:
ax=sns.catplot(data=merged, x='housedsv', y='edlevlsv_numerical', hue='fcstatsv', kind='bar',palette = "Accent")
ax.set(xlabel='Received Housing Services', ylabel='Education Level Numerical')
ax.fig.suptitle('Education level | housing services received')
plt.savefig('img/05_education_level_by_housedsv.png', bbox_inches='tight')

In [ ]:
ax=sns.catplot(data=merged, x='silsv', y='edlevlsv_numerical', hue='fcstatsv', kind='bar',palette = "Accent")
ax.set(ylabel='Education Level Numerical')
ax.fig.suptitle('Education level | special education services received')
plt.savefig('img/04_education_level_by_specedsv.png', bbox_inches='tight')

In [ ]:
ax=sns.catplot(data=merged, x='mentorsv', y='edlevlsv_numerical', hue='fcstatsv', kind='bar',palette = "Accent")
ax.set(xlabel='Received Mentorship Services', ylabel='Education Level Numerical')
ax.fig.suptitle('Education level | mentorship services received')
plt.savefig('img/06_education_level_by_mentorsv.png', bbox_inches='tight')

In [ ]:
fostercare_ohe = pd.read_csv('processed/fostercare_ohe.csv')
fostercare_ohe.head(1)

In [ ]:
fostercare_ohe.index

In [ ]:
dat = fostercare_ohe.pivot_table("highedcert_high school or ged","outcmfcs_yes","medicaid_yes")
ax=sns.heatmap(dat, annot=True,fmt="g", cmap=cmap_blueish)
ax.set(xlabel='Participant has Medicaid', ylabel='Foster Care Status At Outcomes Collection')
ax.set_title('Highest Educational Certification Received is high school or GED')
plt.savefig('img/07_highedcert_high school_or_ged_medicaid.png', bbox_inches='tight')

In [ ]:
dat = fostercare_ohe.pivot_table("highedcert_high school or ged","outcmfcs_yes","cnctadult_yes")
ax=sns.heatmap(dat, annot=True,fmt="g", cmap=cmap_blueish)
ax.set(xlabel='Participant has Connection to Adult', ylabel='Foster Care Status At Outcomes Collection')
ax.set_title('Highest Educational Certification Received is high school or GED')
plt.savefig('img/08_highedcert_high school_or_ged_cnctadult.png', bbox_inches='tight')

In [ ]:
dat = fostercare_ohe.pivot_table("highedcert_high school or ged","outcmfcs_yes","socsecrty_yes")
ax=sns.heatmap(dat, annot=True,fmt="g", cmap=cmap_blueish)
ax.set(xlabel='Social Security', ylabel='Foster Care Status At Outcomes Collection')
ax.set_title('Highest Educational Certification Received is high school or GED')
plt.savefig('img/09_highedcert_high school_or_ged_SocSecrty_yes.png', bbox_inches='tight')

In [ ]:
dat = fostercare_ohe.pivot_table("highedcert_high school or ged","outcmfcs_yes","children_yes")
ax=sns.heatmap(dat, annot=True,fmt="g", cmap=cmap_blueish)
ax.set(xlabel='Children', ylabel='Foster Care Status At Outcomes Collection')
ax.set_title('Highest Educational Certification Received is high school or GED')
plt.savefig('img/10_highedcert_high school_or_ged_children_yes.png', bbox_inches='tight')

In [ ]:
dat = fostercare_ohe.pivot_table("highedcert_high school or ged","outcmfcs_yes","subabuse_yes")
ax=sns.heatmap(dat, annot=True,fmt="g", cmap=cmap_blueish)
ax.set(xlabel='Substance Abuse Referral', ylabel='Foster Care Status At Outcomes Collection')
ax.set_title('Highest Educational Certification Received is high school or GED')
plt.savefig('img/11_highedcert_high school_or_ged_subabuse.png', bbox_inches='tight')

In [ ]:
dat = fostercare_ohe.pivot_table("highedcert_high school or ged","outcmfcs_yes","educaid_yes")
ax=sns.heatmap(dat, annot=True,fmt="g", cmap=cmap_blueish)
ax.set(xlabel='Educational Aid', ylabel='Foster Care Status At Outcomes Collection')
ax.set_title('Highest Educational Certification Received is high school or GED')
plt.savefig('img/12_highedcert_high school_or_ged_educaid.png', bbox_inches='tight')

In [ ]:
dat = fostercare_ohe.pivot_table("highedcert_high school or ged","outcmfcs_yes","homeless_yes")
ax=sns.heatmap(dat, annot=True,fmt="g", cmap=cmap_blueish)
ax.set(xlabel='Homeless', ylabel='Foster Care Status At Outcomes Collection')
ax.set_title('Highest Educational Certification Received is high school or GED')
plt.savefig('img/13_highedcert_high school_or_ged_homeless.png', bbox_inches='tight')

ValueError: 'bugn' is not a valid value for name; supported values are 'Accent', 'Accent_r', 'Blues', 'Blues_r', 'BrBG', 'BrBG_r', 'BuGn', 'BuGn_r', 'BuPu', 'BuPu_r', 'CMRmap', 'CMRmap_r', 'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r', 'Greens', 'Greens_r', 'Greys', 'Greys_r', 'OrRd', 'OrRd_r', 'Oranges', 'Oranges_r', 'PRGn', 'PRGn_r', 'Paired', 'Paired_r', 'Pastel1', 'Pastel1_r', 'Pastel2', 'Pastel2_r', 'PiYG', 'PiYG_r', 'PuBu', 'PuBuGn', 'PuBuGn_r', 'PuBu_r', 'PuOr', 'PuOr_r', 'PuRd', 'PuRd_r', 'Purples', 'Purples_r', 'RdBu', 'RdBu_r', 'RdGy', 'RdGy_r', 'RdPu', 'RdPu_r', 'RdYlBu', 'RdYlBu_r', 'RdYlGn', 'RdYlGn_r', 'Reds', 'Reds_r', 'Set1', 'Set1_r', 'Set2', 'Set2_r', 'Set3', 'Set3_r', 'Spectral', 'Spectral_r', 'Wistia', 'Wistia_r', 'YlGn', 'YlGnBu', 'YlGnBu_r', 'YlGn_r', 'YlOrBr', 'YlOrBr_r', 'YlOrRd', 'YlOrRd_r', 'afmhot', 'afmhot_r', 'autumn', 'autumn_r', 'binary', 'binary_r', 'bone', 'bone_r', 'brg', 'brg_r', 'bwr', 'bwr_r', 'cividis', 'cividis_r', 'cool', 'cool_r', 'coolwarm', 'coolwarm_r', 'copper', 'copper_r', 'crest', 'crest_r', 'cubehelix', 'cubehelix_r', 'flag', 'flag_r', 'flare', 'flare_r', 'gist_earth', 'gist_earth_r', 'gist_gray', 'gist_gray_r', 'gist_heat', 'gist_heat_r', 'gist_ncar', 'gist_ncar_r', 'gist_rainbow', 'gist_rainbow_r', 'gist_stern', 'gist_stern_r', 'gist_yarg', 'gist_yarg_r', 'gnuplot', 'gnuplot2', 'gnuplot2_r', 'gnuplot_r', 'gray', 'gray_r', 'hot', 'hot_r', 'hsv', 'hsv_r', 'icefire', 'icefire_r', 'inferno', 'inferno_r', 'jet', 'jet_r', 'magm...

In [ ]:
sns.catplot(data=merged, x='famsuppsv', y='edlevlsv_numerical', hue='fcstatsv', kind='bar')    # x=hlthedsv, housedsv?

In [ ]:
ax=sns.catplot(data=df, x='raceethn', y='edlevlsv_numerical', hue='fcstatsv', kind='bar')    # x=hlthedsv, housedsv?
ax.set_xticklabels(rotation=90)
# ax.tick_params(axis='x', rotation=90)


In [ ]:
ax=sns.catplot(data=df, x='raceethn', y='edlevlsv_numerical', hue='sex', kind='bar')    # x=hlthedsv, housedsv?
ax.set_xticklabels(rotation=40)

In [ ]:
sns.boxplot(x ='edlevlsv', y ='edlevlsv', data = df, hue ='sex')

In [ ]:
sns.boxplot(x ='medicaid', y ='', data = df, hue ='')

In [ ]:
sns.lineplot(x='dob.1_dt',y ="edlevlsv_numerical", data = merged, ci = 'sd')

In [ ]:
# sns.lineplot(x='emplysklls_yes',y ="highedcert_high school or ged", data = df, errorbar=('ci', 95))

sns.scatterplot(
    data=df, x="age", y="highedcert_high school or ged"
)

In [ ]:
sns.scatterplot(x='sex_female',y ="highedcert_bachelor degree", data = df)

# Models

In [ ]:
# Make two classification models: 
#   - Foster care with demogs
#   - Foster care without demogs

In [ ]:
# Maybe another classification model for ILS (Indep Living Services) 
# Linear Regression
filename = 'processed/fostercare_ohe_train.csv'
df = pd.read_csv(filename)
df.head(1)

In [ ]:

target = ['highedcert_associate degree', 
            'highedcert_bachelor degree', 
            'highedcert_high school or ged', 
            'highedcert_higher degree',
            'highedcert_none of the above',
            'highedcert_vocational certificate',
            'highedcert_vocational license']
x=df.drop(columns=target, axis=1).values
y=df[target].values



In [ ]:
model = LinearRegression()
model.fit(x, y)
print(f'Linear Regression: {model.score(x, y)}')

In [ ]:
# Random forest

In [ ]:
# see distribution

In [ ]:
data = pd.read_csv("combined_ohe_17.csv")
data.head()